In [1]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
from scrape_sofa import get_sofascore_api_data, init_driver, get_all_players_stats_fbref, get_all_teams_stats_fbref
from funciones import get_stat
import pandas as pd

today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [94]:
df_partidos_premier = pd.read_sql(f"SELECT * FROM partido where fecha <= '{yesterday}' AND temporada = 1", engine)

In [184]:
driver = init_driver()
id_partido = 14025013
odds = get_sofascore_api_data(driver, path=f'api/v1/event/{id_partido}/odds/1/all')

In [198]:
def get_stat(stat_list, stat_name, default=None):
    return next((item for item in stat_list if item['marketName'] == stat_name), default)

asian = get_stat(odds['markets'], "Corners 2-Way", default={"home": 0, "away": 0})

resultado = asian['choices'][0]['winning']
resultado

True

In [203]:
odds['markets'][17]

{'sourceId': 179844716,
 'structureType': 1,
 'marketId': 6,
 'marketName': 'First team to score',
 'isLive': False,
 'fid': 179844716,
 'suspended': False,
 'id': 2131949145,
 'marketGroup': 'First team to score',
 'marketPeriod': 'Full-time',
 'choices': [{'initialFractionalValue': '8/15',
   'fractionalValue': '8/15',
   'sourceId': 1464866749,
   'name': 'Manchester City',
   'winning': True,
   'change': 0},
  {'initialFractionalValue': '16/1',
   'fractionalValue': '16/1',
   'sourceId': 1464866747,
   'name': 'No goal',
   'winning': False,
   'change': 0},
  {'initialFractionalValue': '6/4',
   'fractionalValue': '7/5',
   'sourceId': 1464866748,
   'name': 'Manchester United',
   'winning': False,
   'change': -1}]}

In [204]:




driver = init_driver()
ganancias_globales = []
no_disponible = []
for partido in df_partidos_premier['url_sofascore']:
    id_partido = partido.split(':')[2]
    print(f"Scrapeando {id_partido}")
    odds = get_sofascore_api_data(driver, path=f'api/v1/event/{id_partido}/odds/1/all')
    try: 
        asian = get_stat(odds['markets'], "First team to score", default={"home": 0, "away": 0})
        momios = []
    
        for i in range(2):
            resultado = asian['choices'][i]['winning']
            numerador = asian['choices'][i]['fractionalValue'].split('/')[0]
            denominador = asian['choices'][i]['fractionalValue'].split('/')[1]
            numerador = int(numerador)
            denominador = int(denominador)
            multiplicador = 1 + (numerador/denominador)
            momios.append(multiplicador)

        for i in range(len(momios)):
            if momios[i] == min(momios):
                resultado = asian['choices'][i]['winning']
                if resultado == 1: 
                    wager = 10
                    multiplicador = momios[i]
                    ganancias = (wager * multiplicador) - wager
                    print(f"Si hubieras apostado ${wager} hubieras ganado {ganancias}")
                    ganancias_globales.append(ganancias)
                else:
                    wager = 10
                    multiplicador = 0
                    ganancias = (wager * multiplicador) - wager
                    print(f"Ganancias: {ganancias}")
                    ganancias_globales.append(ganancias)
    except KeyError:
        no_disponible.append(id_partido)
        print("No hay momios disponibles para este partido")

print(f"Partidos no disponibles: {no_disponible}")
print(f"Total de ganancias: {sum(ganancias_globales)}")



Scrapeando 14025013
Si hubieras apostado $10 hubieras ganado 3.6363636363636367
Scrapeando 14025016
Ganancias: -10
Scrapeando 14025020
Si hubieras apostado $10 hubieras ganado 6.153846153846153
Scrapeando 14025038
Si hubieras apostado $10 hubieras ganado 12.5
Scrapeando 14025047
Si hubieras apostado $10 hubieras ganado 4.444444444444445
Scrapeando 14025050
Ganancias: -10
Scrapeando 14025053
Ganancias: -10
Scrapeando 14025028
Si hubieras apostado $10 hubieras ganado 6.153846153846153
Scrapeando 14025056
Ganancias: -10
Scrapeando 14025059
Si hubieras apostado $10 hubieras ganado 8.333333333333336
Scrapeando 14025088
Si hubieras apostado $10 hubieras ganado 15.0
Scrapeando 14025083
Ganancias: -10
Scrapeando 14025062
Si hubieras apostado $10 hubieras ganado 5.7142857142857135
Scrapeando 14025068
Si hubieras apostado $10 hubieras ganado 12.0
Scrapeando 14025071
Si hubieras apostado $10 hubieras ganado 9.090909090909093
Scrapeando 14025065
No hay momios disponibles para este partido
Scrapean

In [111]:
driver = init_driver()
ganancias_globales = []
no_disponible = []
for partido in df_partidos_premier['url_sofascore']:
    id_partido = partido.split(':')[2]
    print(f"Scrapeando {id_partido}")
    odds = get_sofascore_api_data(driver, path=f'api/v1/event/{id_partido}/odds/1/all')
    momios = []
    try: 
        resultados = []
        for i in range(2):
            resultado = odds['markets'][3]['choices'][i]['winning']
            resultados.append(resultado)
            numerador = odds['markets'][3]['choices'][i]['fractionalValue'].split('/')[0]
            denominador = odds['markets'][3]['choices'][i]['fractionalValue'].split('/')[1]
            numerador = int(numerador)
            denominador = int(denominador)
            multiplicador = 1 + (numerador/denominador)
            momios.append(multiplicador)

        


        
        if momios[0] == min(momios):
            resultado = odds['markets'][3]['choices'][0]['winning']
            otro_resultado = odds['markets'][3]['choices'][1]['winning']
            if resultado == 1: 
                wager = 10
                multiplicador = momios[0]
                ganancias = (wager * multiplicador) - wager
                print(f"Si hubieras apostado ${wager} hubieras ganado {ganancias}")
                ganancias_globales.append(ganancias)
            elif (resultado == 0 and otro_resultado == 0):
                ganancias = 0
                print("Apuesta anulada, el partido acabo en empate")
                ganancias_globales.append(ganancias)
            
            else:
                wager = 10
                multiplicador = 0
                ganancias = (wager * multiplicador) - wager
                print(f"Ganancias: {ganancias}")
                ganancias_globales.append(ganancias)
        else: 
            resultado = odds['markets'][3]['choices'][1]['winning']
            otro_resultado = odds['markets'][3]['choices'][1]['winning']
            if resultado == 1: 
                wager = 10
                multiplicador = momios[1]
                ganancias = (wager * multiplicador) - wager
                print(f"Si hubieras apostado ${wager} hubieras ganado {ganancias}")
                ganancias_globales.append(ganancias)
            elif (resultado == 0 and otro_resultado == 0):
                ganancias = 0
                print("Apuesta anulada, el partido acabo en empate")
                ganancias_globales.append(ganancias)
            
            else:
                wager = 10
                multiplicador = 0
                ganancias = (wager * multiplicador) - wager
                print(f"Ganancias: {ganancias}")
                ganancias_globales.append(ganancias)

    except KeyError:
        no_disponible.append(id_partido)
        print("No hay momios disponibles para este partido")

print(f"Partidos no disponibles: {no_disponible}")
print(f"Total de ganancias: {sum(ganancias_globales)}")



Scrapeando 14025013
Si hubieras apostado $10 hubieras ganado 1.1111111111111107
Scrapeando 14025016
Apuesta anulada, el partido acabo en empate
Scrapeando 14025020
Apuesta anulada, el partido acabo en empate
Scrapeando 14025038
Apuesta anulada, el partido acabo en empate
Scrapeando 14025047
Si hubieras apostado $10 hubieras ganado 1.8181818181818183
Scrapeando 14025050
Si hubieras apostado $10 hubieras ganado 1.8181818181818183
Scrapeando 14025053
Apuesta anulada, el partido acabo en empate
Scrapeando 14025028
Si hubieras apostado $10 hubieras ganado 3.6363636363636367
Scrapeando 14025056
Si hubieras apostado $10 hubieras ganado 4.444444444444445
Scrapeando 14025059
Si hubieras apostado $10 hubieras ganado 5.7142857142857135
Scrapeando 14025088
Si hubieras apostado $10 hubieras ganado 2.857142857142856
Scrapeando 14025083
Ganancias: -10
Scrapeando 14025062
Si hubieras apostado $10 hubieras ganado 3.0
Scrapeando 14025068
Apuesta anulada, el partido acabo en empate
Scrapeando 14025071
Si